In [2]:
import cv2
import numpy as np

In [3]:
confThreshold=0.25
nmsThreshold=0.40
inpWidth=416
inpHeight=416
risk_count=0
low_risk_count=0
safe_count=0

In [4]:
def how_close(x1,y1,x2,y2):
    if dist(x1,y1,x2,y2)>15 and dist(x1,y1,x2,y2)<150:
        return 1
    elif dist(x1,y1,x2,y2)>=150 and dist(x1,y1,x2,y2)<=300:
        return 2
    elif dist(x1,y1,x2,y2)>300:
        return 3

In [5]:
def dist(x1,y1,x2,y2):
    dis=((x1-x2)**2 + (y1-y2)**2) ** 0.5
    return dis

In [6]:
def getOutputsNames(net):
    layerNames=net.getLayerNames()
    return [layerNames[i[0]-1] for i in net.getUnconnectedOutLayers()]

In [7]:
def drawPred(classId,conf,left,top,right,bottom):
    cv2.rectangle(frame,(left,top),(right,bottom),(255,178,50),3)

In [8]:
def postprocess(frame,outs):
    frameHeight=frame.shape[0]
    frameWidth=frame.shape[1]
    classIDs=[]
    confidences=[]
    boxes=[]
    center=[]
    status=[]
    
    for out in outs:
        for detection in out:
            scores=detection[5:]
            classID=np.argmax(scores)
            confidence=scores[classID]

            if classes[classID]=='p':
                if confidence>confThreshold:
                    centerX=int(detection[0]*frameWidth)
                    centerY=int(detection[1]*frameHeight)

                    width=int(detection[2]*frameWidth)
                    height=int(detection[3]*frameHeight)

                    left=int(centerX- width/2)
                    top=int(centerY- height/2)

                    classIDs.append(classID)
                    confidences.append(float(confidence))
                    boxes.append([left,top,width,height])
    indices=cv2.dnn.NMSBoxes(boxes,confidences,confThreshold,nmsThreshold)
    
    for i in indices:
        sub_part=frame[frameHeight - 950:frameHeight-720, 20:490]
        black_rect = np.zeros(sub_part.shape, dtype=np.uint8)
        res = cv2.addWeighted(sub_part, 0.90, black_rect, 0.10, 0.5)
        frame[frameHeight - 950:frameHeight-720, 20:490]=res
        high_risk_person=0
        low_risk_person=0
        safe_person=0
        i=i[0]
        box=boxes[i]
        left=box[0]
        top=box[1]
        width=box[2]
        height=box[3]
        if ((left > (x-7.5) and left < (x+7.5)) or (top > (y-7.5) and top < (y+7.5))):
            continue
        else:
            drawPred(classIDs[i],confidences[i],left,top,left+width,top+height)
        center.append([int(left+width/2),int(top+height/2)])
        
        for i in range(len(center)):
            pts=[int(center[i][j]/3) for j in range(2)]
        
        for i in range(len(center)):
            d=how_close(x+w/2,y+h/2,center[i][0],center[i][1])
            if d==1:
                cv2.line(frame,(int(x+w/2),int(y+h/2)),(int(center[i][0]),int(center[i][1])),(0,0,255),2)
                high_risk_person=high_risk_person+1
                if(high_risk_person>0):
                    high_risk_person=high_risk_person
                else:
                    high_risk_person=0
                total_high_risk_count="Risk Count : "+str(high_risk_person)
                cv2.putText(frame, total_high_risk_count,(100,200), cv2.FONT_HERSHEY_TRIPLEX, 1,(0,0,255),4)
            elif d==2:
                cv2.line(frame,(int(x+w/2),int(y+h/2)),(int(center[i][0]),int(center[i][1])),(0,123,255),2)
                low_risk_person=low_risk_person+1
                if(low_risk_person>0):
                    low_risk_person=low_risk_person
                else:
                    low_risk_person=0
                total_low_risk_count="Low Risk Count : "+str(low_risk_person)
                cv2.putText(frame, total_low_risk_count,(100,250), cv2.FONT_HERSHEY_TRIPLEX, 1,(0,123,255),4)
            else:
                cv2.line(frame,(int(x+w/2),int(y+h/2)),(int(center[i][0]),int(center[i][1])),(0,191,0),2)
                safe_person=safe_person+1
                if(safe_person>0):
                    safe_person=safe_person
                else:
                    safe_person=0
                total_safe_count="Safe Count : "+str(safe_person)
                #cv2.putText(frame, total_safe_count,(100,300), cv2.FONT_HERSHEY_TRIPLEX, 1,(0,191,0),4)


In [9]:
classesFile="bin/coco.names"
classes=None

In [10]:
with open(classesFile,'rt') as f:
    classes=f.read().strip("\n")

In [11]:
modelConf="bin/yolov3.cfg"
modelWeights="bin/yolov3.weights"

In [12]:
net=cv2.dnn.readNetFromDarknet(modelConf,modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [13]:
winName='Tracker'
tracker=cv2.TrackerCSRT_create()
black_window=np.zeros((200,333),dtype=np.uint8)
initBB=None
cv2.namedWindow(winName,cv2.WINDOW_NORMAL)
cv2.resizeWindow(winName,1000,600)
cap=cv2.VideoCapture('TownCentreXVID.avi')
while True:
    hasFrame,frame=cap.read()
    if frame is None:
        break
    
    frameHeight=frame.shape[0]
    frameWidth=frame.shape[1]
    if initBB is not None:
        (success,box)=tracker.update(frame)
        
        if success:
            (x,y,w,h)=[int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x + w, y + h),(0, 0, 255), 2)
            a=int((x+w)/6)
            b=int((y+h)/6)
            #print(a,b)
            black_img=cv2.circle(black_window,(a,b),5,(255,255,255),-1)
            black_img=cv2.putText(black_window,"Tracing the path ",(10,160),cv2.FONT_HERSHEY_SIMPLEX, 1,(121,111,255),4)
            cv2.imshow('Black',black_img)
        else :
        # Tracking failure
            cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)
            
    
            
        blob=cv2.dnn.blobFromImage(frame, 1 / 255.0, (inpHeight, inpWidth),swapRB=True, crop=False)

        net.setInput(blob)
        outs=net.forward(getOutputsNames(net))
        postprocess(frame,outs)
    
        
    cv2.imshow(winName,frame)
    
    key = cv2.waitKey(1) & 0xFF
    if (key == ord('s')):
        initBB = cv2.selectROI("Frame",frame, fromCenter=False,showCrosshair=True)
        tracker.init(frame,initBB)
    elif key==ord('q'):
        break
    
    
cap.release()
cv2.destroyAllWindows()